In [ ]:
import requests
import time
import pandas as pd
import re
from datetime import date, timedelta
import json
import io

In [ ]:
!gdown 'https://drive.google.com/uc?id=1A******B8s-*********************************'

Downloading...
From: https://drive.google.com/uc?id=1A-73oB8s-haqYVIYygiPVNrlSy7GjP4X
To: /content/holodilnik-bq-376311-50cc6f8cfc32.json
100% 2.38k/2.38k [00:00<00:00, 3.60MB/s]


In [ ]:
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file('/content/holodilnik-bq-376311-50cc6f8cfc32.json')
load_date = (date.today()-timedelta(days=1)).strftime('%Y-%m-%d')

In [ ]:
dict_ag = pd.DataFrame({'code_agency': ['a485', 'a617', 'a564', 'a339', 'a776', 'a999', 'a499', 'a224', 'a823', 'a662', 'a799', 'a391', 'a888', 'a111'], 
                        'agency_name': ['2leads', 'Borscht', 'Colead', 'GoMobile', 'Gradientt', 'In-house', 'Mediaserfer', 'Mobisharks', 'MobX', 'Rocket10', 'ThinkMobile', 'TopTraffic', 'Mobio', 'xapads']})
dict_s = pd.DataFrame({'code_source': ['c201', 'c202', 'c203', 'c204', 'c210', 'c205', 'c206', 'c209', 'c207', 'c208', 'c211', 'c212', 'c217', 'c215', 'с216', 'с218', 'с219'], 
                        'source_name': ['Apple Search Ads', 'DV360', 'Facebook', 'Google Adwords', 'in-app', 'Mytarget', 'Tik Tok', 'Twitter', 'VK', 'Yandex.Direct', 'Appnext', 'beta vk', 'xapads', 'AdGate', 'Bigo Ads', 'untiy', 'mintegral']})
dict_t = pd.DataFrame({'code_type': ['u1', 'u2'], 'type_name': ['UA', 'RET']})

In [ ]:
fields = 'appmetrica_device_id,tracker_name,notifying_status,install_datetime,attempt_datetime,device_type,os_name,conversion_datetime,event_name,cost_model'
application_id = '4089013'
url= f'https://api.appmetrica.yandex.ru/logs/v1/export/postbacks.json?application_id={application_id}&date_since={load_date}&date_until={load_date}&fields={fields}'
header = {'Authorization': 'OAuth y0_*********************************_6uv*****'}
response = requests.get(url=url, headers=header)
k=0
while response.status_code != 200 and k<90:
  response = requests.get(url=url, headers=header)
  print(response.status_code)
  time.sleep(5)
  k+=1

In [ ]:
js_df = response.json()
df = pd.DataFrame(js_df['data'])
df_rec = df[df.tracker_name.str.contains('realweb', flags=re.IGNORECASE)]

In [ ]:
df_rec.groupby('event_name').agg({'appmetrica_device_id':'count'})

,appmetrica_device_id
event_name,
,3998
PurchaseSuccess,55
payment_by_card,96
payment_in_cash,55
purchase,217


In [ ]:
# df_rec.to_gbq('grow-food-367913.Meta.appmetrica_postback', 
#           project_id='grow-food-367913',  
#           if_exists='append',
#           credentials=credentials)

In [ ]:
sql = """SELECT * FROM `grow-food-367913.Meta.appmetrica_postback`"""
all_data = pd.read_gbq(sql, credentials=credentials, dialect='standard')
all_data = all_data.drop_duplicates()

In [ ]:
# all_data.to_gbq('grow-food-367913.Meta.appmetrica_postback', 
#           project_id='grow-food-367913',  
#           if_exists='replace',
#           credentials=credentials)

In [ ]:
sql = """SELECT * FROM `grow-food-367913.Meta.appmetrica_postback`"""
df = pd.read_gbq(sql, credentials=credentials, dialect='standard')
df.install_datetime = pd.to_datetime(df.install_datetime)
df.attempt_datetime = pd.to_datetime(df.attempt_datetime)
df.conversion_datetime = pd.to_datetime(df.conversion_datetime)
# df = df.drop_duplicates()
df['date'] = df.install_datetime.dt.date
# .query('install_datetime >= "2022-11-21"') \
df_installs = df.groupby(['date', 'tracker_name', 'device_type', 'os_name'], as_index=False) \
        .agg({'appmetrica_device_id': 'nunique'}) \
        .rename(columns={'appmetrica_device_id': 'installs'})
df_events = df.query('event_name == "af_purchase_new"') \
          .groupby(['date', 'tracker_name', 'device_type', 'os_name'], as_index=False) \
          .agg({'appmetrica_device_id': 'nunique'}) \
          .rename(columns={'appmetrica_device_id': 'event_count'})
all_data = df_installs.merge(df_events, how='outer', on=['date', 'tracker_name', 'device_type', 'os_name']).fillna(0)
all_data[['date', 'tracker_name', 'device_type', 'os_name']] = all_data[['date', 'tracker_name', 'device_type', 'os_name']].astype('str')
all_data[['installs', 'event_count']] = all_data[['installs', 'event_count']].astype('float')

In [ ]:
all_data['code_agency'] = all_data.tracker_name.apply(lambda x: x.lower().split('_')[3] if (x.lower().split('_')[2] in ['and', 'ios']) else x.lower().split('_')[4])
all_data['code_source'] = all_data.tracker_name.apply(lambda x: x.lower().split('_')[4] if (x.lower().split('_')[2] in ['and', 'ios']) else x.lower().split('_')[5])
# all_data['os_name'] = all_data.tracker_name.apply(lambda x: 'android' if x.lower().split('_')[2] == 'and' else ('ios' if x.lower().split('_')[2] == 'ios' else x))
merge_data = all_data.merge(dict_ag, how='left', on='code_agency') \
                          .merge(dict_s, how='left', on='code_source') \
                          .drop(columns=['code_agency', 'code_source'])
merge_data.date = pd.to_datetime(merge_data.date.astype('str'))

In [ ]:
from_date_fr = (date.today()-timedelta(days=2)).strftime('%Y-%m-%d')
CHANNEL = 'h****dil***********************U'
USER_KEY = '12*************'
url = f'https://api.fraudscore.ai/{CHANNEL}/install/custom/groups?sort=offer_id_asc&group=offer_id&group=affiliate_id&group=os_family&group=score_str&group=date&field=offer_id&field=any_offer_name&field=affiliate_id&field=any_affiliate_name&field=os_family&field=score_str&field=score_avg'
params = dict(key = USER_KEY)
# params['group'] = ['date','offer_name']
# params['field'] = ['any_affiliate_name']
params['install_filter'] = json.dumps([
	"AND",
	["date_gt", from_date_fr],
	["date_lt", date.today().strftime('%Y-%m-%d')],
    ["score_str", "high"]])

# params['having'] = json.dumps(['AND',["score_high_ge", "1"], ["count_ge", "1"]])
# params['page_size'] = 1000000
r = requests.get(url, params, stream = True)

In [ ]:
r.json()

In [ ]:
date = []
any_offer_name = []
any_affiliate_name = []
score_str = []
custom_users = []
i = 0
for line in r.iter_lines():
    i += 1
    if i == 1 and 'purchase' not in str(line).replace("b'[", "").replace("]'", "").replace('"', '').split(','):
        break 
    elif  i == 1:
        continue
    data = json.loads(line)
    date += [data['date']]
    any_offer_name += [data['any_offer_name']]
    any_affiliate_name += [data['any_affiliate_name']]
    score_str += [data['score_str']]
    custom_users += [data['custom_users'][1] if len(data['custom_users'])!=0 else "0"]
d = {'date': date, 'any_offer_name': any_offer_name, 'any_affiliate_name': any_affiliate_name, 'score_str': score_str, 'custom_users': custom_users}
df_fraud_score = pd.DataFrame(data=d)
df_fr_sc = df_fraud_score.rename(columns={'any_offer_name': 'tracker_name', 'custom_users': 'fraud'}).query('any_affiliate_name == "RealWeb" and score_str == "high"')[['date', 'tracker_name', 'fraud']]
df_fr_sc.fraud = df_fr_sc.fraud.astype('int')
df_agg = df_fr_sc.query('fraud > 0')

In [ ]:
df_agg

,date,tracker_name,fraud
1,2023-04-02,Realweb_inapp_Android_CPA_Adshustle_a224,1


In [ ]:
# df_agg.to_gbq('grow-food-367913.Meta.data_fraud', 
#           project_id='grow-food-367913',  
#           if_exists='append',
#           credentials=credentials)

In [ ]:
sql_fraud = """SELECT * FROM `grow-food-367913.Meta.data_fraud`"""
all_data_fraud = pd.read_gbq(sql_fraud, credentials=credentials, dialect='standard')
all_data_fraud = all_data_fraud.drop_duplicates()

In [ ]:
# all_data_fraud.to_gbq('grow-food-367913.Meta.data_fraud', 
#           project_id='grow-food-367913',  
#           if_exists='replace',
#           credentials=credentials)

In [ ]:
import json
CHANNEL = '***************************'
USER_KEY = '***************************'
EVENT = 'install'
url = f'https://get.fraudscore.ai/{CHANNEL}/{EVENT}/groups'
params = dict(key = USER_KEY)
params['group'] = ['date','offer_name']
params['field'] = ['any_affiliate_name']
params['install_filter'] = json.dumps([
	"AND",
	["date_gt", "2023-03-01"],
	["date_lt", "2023-03-31"]])
params['having'] = json.dumps(['AND',["score_high_ge", "1"], ["count_ge", "1"]])
params['page_size'] = 1000000
r = requests.get(url, params, stream = True)
date = []
offer_name = []
any_affiliate_name = []
count = []
score_high = []
for line in r.iter_lines():
  data = json.loads(line)
  date += [data['date']]
  offer_name += [data['offer_name']]
  any_affiliate_name += [data['any_affiliate_name']]
  count += [data['count']]
  score_high += [data['score_high']]  
d = {'date': date, 'offer_name': offer_name, 'any_affiliate_name': any_affiliate_name, 'count': count, 'score_high': score_high}
df_fraud_score = pd.DataFrame(data=d)
df_fr_sc = df_fraud_score.rename(columns={'offer_name': 'tracker_name'}).query('(date>="2023-03-01") and(date<="2023-03-31")')[['date', 'tracker_name', 'count', 'score_high']]
df_fr_sc.date = df_fr_sc.date.astype('str')
df_fr_sc.tracker_name = df_fr_sc.tracker_name.astype('str')
df_fr_sc['count'] = df_fr_sc['count'].astype('int')
df_fr_sc.score_high = df_fr_sc.score_high.astype('int')

In [ ]:
df_fr_sc

,date,tracker_name,count,score_high
12,2023-03-22,Realweb_inapp_Valuadle_Android_a799,1,1
28,2023-03-29,Realweb_inapp_Android_CPA_Adshustle_a224,753,15
30,2023-03-10,Realweb_inapp_Kvidu2_Android_a224,7,1
35,2023-03-25,Realweb_inapp_Android_CPA_Kvidu_a224,12,3
36,2023-03-22,Realweb_inapp_Android_CPA_Joinship_a224,9,1
...,...,...,...,...
2410,2023-03-10,Realweb_inapp_Touchpoint_Android_a799,30,17
2414,2023-03-12,Realweb_inapp_Mustang_Android_a799,27,6
2417,2023-03-06,Realweb_inapp_Touchpoint_Android_a799,34,9
2420,2023-03-03,Realweb_inapp_Roockmobile_Android_a224,12,3


In [ ]:
df_fr_sc['score_high'].sum()

1542

In [ ]:
# df_fr_sc.to_gbq('grow-food-367913.Meta.data_fraud_install', 
#           project_id='grow-food-367913',  
#           if_exists='replace',
#           credentials=credentials)

In [ ]:
sql_fraud_install = """SELECT * FROM `grow-food-367913.Meta.data_fraud_install`"""
all_inst_fraud = pd.read_gbq(sql_fraud_install, credentials=credentials, dialect='standard')
all_inst_fraud = all_inst_fraud.drop_duplicates()

In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from gspread_dataframe import set_with_dataframe

scope = [
    'https://www.googleapis.com/auth/spreadsheets',
]

GOOGLE_KEY_FILE = '/content/grow-food-367913-2a42be2fc3c2.json'

credentials_spread = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_KEY_FILE, scope)
gc = gspread.authorize(credentials_spread)

workbook_key = '1_92jon59KaQE2eQps2PYgVYIIvcHnbM_lNfRCgNcdzo'
workbook = gc.open_by_key(workbook_key)

In [ ]:
sheet2 = workbook.worksheet('budget')
values2 = sheet2.get_all_values()
data2 = pd.DataFrame(values2[1:],columns=values2[0])

In [ ]:
all_data_fraud.head()

,date,tracker_name,fraud
0,2023-02-07,Realweb_ Offersinfinite_AND_a799_c210_m405_u1,12
1,2023-02-07,Realweb_iconpeak_AND_a799_c210_m405_u1,1
2,2023-02-07,Realweb_metmobi_AND_a799_c210_m405_u1,1
3,2023-02-06,Realweb_valuadle_AND_a799_c210_m405_u1,1
4,2023-02-06,Realweb_ Offersinfinite_AND_a799_c210_m405_u1,1


In [ ]:
# data2.start_date = pd.to_datetime(data2.start_date, format = '%Y-%m-%d')
# data2.end_date = pd.to_datetime(data2.end_date, format = '%Y-%m-%d')
all_inst_fraud['date_agg'] = pd.to_datetime(all_inst_fraud.date.astype('str'))
all_data_fraud['date_agg'] = pd.to_datetime(all_data_fraud.date.astype('str'))
all_inst_fraud['code_agency'] = all_inst_fraud.tracker_name.apply(lambda x: x.lower().split('_')[3] if (x.lower().split('_')[2] in ['and', 'ios']) else x.lower().split('_')[4])
all_inst_fraud['code_source'] = all_inst_fraud.tracker_name.apply(lambda x: x.lower().split('_')[4] if (x.lower().split('_')[2] in ['and', 'ios']) else x.lower().split('_')[5])
all_data_fraud['code_agency'] = all_data_fraud.tracker_name.apply(lambda x: x.lower().split('_')[3] if (x.lower().split('_')[2] in ['and', 'ios']) else x.lower().split('_')[4])
all_data_fraud['code_source'] = all_data_fraud.tracker_name.apply(lambda x: x.lower().split('_')[4] if (x.lower().split('_')[2] in ['and', 'ios']) else x.lower().split('_')[5])
all_inst_fraud['os_name'] = all_inst_fraud.tracker_name.apply(lambda x: 'android' if x.lower().split('_')[2] == 'and' else ('ios' if x.lower().split('_')[2] == 'ios' else x))
all_data_fraud['os_name'] = all_data_fraud.tracker_name.apply(lambda x: 'android' if x.lower().split('_')[2] == 'and' else ('ios' if x.lower().split('_')[2] == 'ios' else x))
merge_data_inst = all_inst_fraud.merge(dict_ag, how='left', on='code_agency') \
                          .merge(dict_s, how='left', on='code_source') \
                          .drop(columns=['code_agency', 'code_source'])
merge_data_fraud = all_data_fraud.merge(dict_ag, how='left', on='code_agency') \
                          .merge(dict_s, how='left', on='code_source') \
                          .drop(columns=['code_agency', 'code_source'])
intsalls = []
events = []
fraud_ev = []
fraud_inst = []
for i in data2.itertuples():
    start_date, end_date, partner, platform, source  = i.start_date, i.end_date, i.partner, i.platform, i.source
    intsalls += [merge_data.query('date >= @start_date and date <= @end_date and os_name == @platform and agency_name == @partner and source_name == @source').installs.sum()]
    events += [merge_data.query('date >= @start_date and date <= @end_date and os_name == @platform and agency_name == @partner and source_name == @source').event_count.sum()]
    fraud_ev += [merge_data_fraud.query('date_agg >= @start_date and date_agg <= @end_date and os_name == @platform and agency_name == @partner and source_name == @source').fraud.sum()]
    fraud_inst += [merge_data_inst.query('date_agg >= @start_date and date_agg <= @end_date and os_name == @platform and agency_name == @partner and source_name == @source').score_high.sum()]
data2['intsalls'] = pd.Series(intsalls)
data2['events'] = pd.Series(events)
data2['fraud_ev'] = pd.Series(fraud_ev)
data2['fraud_inst'] = pd.Series(fraud_inst)
data2['real_intsalls'] = data2.intsalls - data2.fraud_inst
data2['real_events'] = data2.events - data2.fraud_ev
data2 = data2.drop(columns=['intsalls', 'events', 'fraud_ev', 'fraud_inst'])
all_inst_fraud = all_inst_fraud.drop(columns=['date_agg', 'code_agency', 'code_source', 'os_name'])
all_data_fraud = all_data_fraud.drop(columns=['date_agg', 'code_agency', 'code_source', 'os_name'])

In [ ]:
# data1.plan_budget = data1.plan_budget.apply(lambda x: str(x).replace(',', '.').replace('\xa0', '')).astype('float')
# data1.plan_installs = data1.plan_installs.apply(lambda x: str(x).replace(',', '.').replace('\xa0', '')).astype('float')
# data1.event_af_purchase_new = data1.event_af_purchase_new.apply(lambda x: str(x).replace(',', '.').replace('\xa0', '')).astype('float')
data2.rate_for_us = data2.rate_for_us.apply(lambda x: str(x).replace(',', '.').replace('\xa0', '')).astype('float')
data2.rate_for_parthner = data2.rate_for_parthner.apply(lambda x: str(x).replace(',', '.').replace('\xa0', '')).astype('float')
data2.plan_af_purchase_new = data2.plan_af_purchase_new.apply(lambda x: str(x).replace(',', '.').replace('\xa0', '')).astype('float')
data2['real_budget'] = data2.real_events * data2.rate_for_us
data2['marga'] = (data2.rate_for_us - data2.rate_for_parthner)*data2.real_events
data2['plan_budget'] = data2.rate_for_us * data2.plan_af_purchase_new

In [ ]:
sheet3 = workbook.worksheet('real_budget')
sheet3.clear()
set_with_dataframe(worksheet=sheet3, dataframe=data2, include_index=False,
include_column_header=True, resize=True)

In [ ]:
# df_data['os_name'] = df_data.tracker_name.apply(lambda x: 'android' if x.lower().split('_')[2] == 'and' else ('ios' if x.lower().split('_')[2] == 'ios' else x))
data_for_report = all_data.groupby(['date', 'tracker_name', 'os_name'], as_index=False) \
          .agg({'installs': 'sum', 'event_count': 'sum'}) \
          .merge(all_data_fraud, how='left', on=['date', 'tracker_name']) \
          .merge(all_inst_fraud, how='left', on=['date', 'tracker_name']) \
          .rename(columns={'count': 'conv_count', 'score_high': 'fraud_inst'}).fillna(0)
data_for_report.fraud = data_for_report.fraud.fillna(0).astype('int')
data_for_report.installs = data_for_report.installs.astype('int')
data_for_report.event_count = data_for_report.event_count.astype('int')
data_for_report.date = pd.to_datetime(data_for_report.date)
data_report = data_for_report.query('date >= "2022-11-21"')
data_report.date = data_report.date.dt.date.astype('str')
data_report

In [ ]:
# data_report.to_gbq('grow-food-367913.Meta.data_for_report', 
#           project_id='grow-food-367913',  
#           if_exists='replace',
#           credentials=credentials)